In [ ]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;

In [ ]:
#'''
# Creating Results Files ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

file_summary = open("Rel_Results_Summary.csv", "w");

file_summary.write('Instance\t, Budget\t, Cal_A_Level\t, Nodes\t, Arcs\t, OptGap\t,');
file_summary.write('Obj_value\t, Flow\t, Level_1\t, Level_2\t, Level_3\t, Other_Levels\t, Run_Time\n');
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#'''

num = 5;

model = {};
budget = [0, 5, 10];
rate = [1];

T_Lim = 30;

network = {};


# Input -- Reading the networks
for n in range(num):
    model[n] = {};
    network[n] = 'Net'+str(n+1);
     
    for b in range(len(budget)):  
        model[n][b] = {};
        
        for r in range(len(rate)):
            print('\n');
            print(network[n], ', budget', budget[b]);
            
            start_time = time.time(); 
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
                calA_level = int(row1[2]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);                        
                                           
                    
                A = 0;
                U = G.edges[t,s]['capacity'];

                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;                     
            
            M = A;
            Md = A;
            Mt = A;

            # Tau for the linearization of x_{ts}*delta
            tau = range(math.ceil(math.log(U))+1); 


            model[n][b][r] = gp.Model("Pessimistic_Relaxed"); 
            
            eta = model[n][b][r].addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
            gamma = model[n][b][r].addVars(G.edges, vtype=GRB.BINARY);
            xBar = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            x = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            z = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            alpha = model[n][b][r].addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY); 
            theta = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
            pi = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb =-GRB.INFINITY, ub=0);
            omega = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            
            
            delta = model[n][b][r].addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub=0);                               


            v = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
            muTau = model[n][b][r].addVars(tau, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
            mu = model[n][b][r].addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);  
            lamb = model[n][b][r].addVars(tau, vtype=GRB.BINARY);        
                
            
            # Constraints
            
            # Upper-Level
            model[n][b][r].addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget[b]);   
            
            model[n][b][r].addConstrs(gp.quicksum(xBar[j,i] for i in G.successors(j))
                 - gp.quicksum(xBar[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
            model[n][b][r].addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))
                 - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    
            model[n][b][r].addConstr(xBar[t,s] == gp.quicksum(pow(2, tau[u])*lamb[u] for u in tau));
            
            for i,j in G.edges:
                model[n][b][r].addConstr(xBar[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);
                model[n][b][r].addConstr(x[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);
                
                model[n][b][r].addConstr(v[i,j] <= theta[i,j]);    
                model[n][b][r].addConstr(v[i,j] >= theta[i,j] - Mt*gamma[i,j]);
                model[n][b][r].addConstr(v[i,j] <= Mt*(1-gamma[i,j]));
                if G.edges[i,j]['special'] == 1:
                    model[n][b][r].addConstr(alpha[i] - alpha[j] + theta[i,j] + pi[i,j]*G.edges[i,j]['special'] >= 0); 
                    model[n][b][r].addConstr(x[i,j] - (1/M)*z[i,j]*G.edges[i,j]['special'] >= 0);
                    model[n][b][r].addConstr(omega[i,j]*G.edges[i,j]['special'] - (1/M)*pi[i,j]*G.edges[i,j]['special'] >= 1);
                elif i!=t:
                    model[n][b][r].addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0); 
                    
            
            model[n][b][r].addConstr(x[t,s] >= xBar[t,s]);                                                                                                                    
    
            model[n][b][r].addConstr(alpha[t] - alpha[s] + theta[t,s] + delta >= 0);                                    
    
            model[n][b][r].addConstr(mu == gp.quicksum(pow(2, tau[u])*muTau[u] for u in tau));
        
            for u in tau:
                model[n][b][r].addConstr(muTau[u] >= delta);
                model[n][b][r].addConstr(muTau[u] >= -Md*lamb[u]);
                model[n][b][r].addConstr(muTau[u] <= delta - Md*(lamb[u] - 1));
            
            
            model[n][b][r].addConstr(eta >= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges)); 
    
            model[n][b][r].addConstr(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges) 
                                     >= gp.quicksum(G.edges[i,j]['capacity']*v[i,j] for i,j in G.edges) + 
                                     gp.quicksum(omega[i,j]*G.edges[i,j]['special'] for i,j in G.edges) + mu);
            
            
            model[n][b][r].setObjective(eta, GRB.MINIMIZE); 
            
            #model[n][b][r].setParam("IntegralityFocus",1);
            #model[n][b][r].setParam("NumericFocus",2);
            
            #model[n][b][r].setParam('TimeLimit', T_Lim);
            
            model[n][b][r].update();
            model[n][b][r].setParam("OutputFlag", 0);
            model[n][b][r].optimize();
            
            
            
            
            # ~~~~~~~~~~~~~~~ Printing Results ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            file = open('Rel_results_'+network[n]+'_b'+str(budget[b])+'.txt', "w");
            
            file.write('Number of Nodes: %g' % (t+1) +'\n');
            file.write('Number of Arcs: %g' % G.number_of_edges() +'\n');
            file.write('Budget: %g' % budget[b] +'\n\n');
            
            if model[n][b][r].status == GRB.status.OPTIMAL:
                print('Solved to Optimality \n');
                
                print('xBar_ts: ', xBar[t,s].x);
                
                print('x_ts: ', x[t,s].x);
                
                
                print("\n");
                for i,j in G.edges:
                    if gamma[i,j].x > 0.0:
                        print('gamma: ',i,j, gamma[i,j].x);
                
                print("\n");
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        if x[i,j].x > 0.0:
                            print('x: ',i,j, x[i,j].x);
                
                print("\n");
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        if z[i,j].x*G.edges[i,j]['special'] == 0:
                            print('z: ',i,j, z[i,j].x*G.edges[i,j]['special']);
                
                '''
                print("\n");
                for i,j in G.edges:
                    if theta[i,j].x > 0.0:
                        print('Theta: ',i,j, theta[i,j].x);
                
                print("\n");
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        if pi[i,j].x*G.edges[i,j]['special'] != 0:
                            print('pi: ',i,j, pi[i,j].x*G.edges[i,j]['special']);
                
                print("\n");
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        if omega[i,j].x*G.edges[i,j]['special'] == 0:
                            print('Omega: ',i,j, omega[i,j].x*G.edges[i,j]['special']);
                '''
                
                '''
                print("\n");
                for i in G.nodes:
                    print('alpha: ',i, ", ", alpha[i].x);
                '''
                
                               
                obj = model[n][b][r].objVal;
                OptGap = 0.0;
                sol = A - obj;
                file.write('Solved to Optimality\n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % obj +'\n');
                
            elif model[n][b][r].status == GRB.status.TIME_LIMIT:
                print('\nReached Time Limit');
                for i,j in G.edges:
                    if gamma[i,j].x > 0.0:
                        print('gamma: ',i,j, gamma[i,j].x);
                
                obj = model[n][b][r].objVal;
                LB = model[n][b][r].objBound;
                sol = A - obj;
                file.write('Reached Time Limit \n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % obj +'\n');
                file.write('UB: %g' % obj +'\n'); 
                file.write('LB: %g' % LB +'\n');
                OptGap = (obj-LB)/obj;
                file.write('Opt Gap: %s' % str("{:.2f}".format((OptGap)))+'\n');
            
            
            file.write('Max-Flow: %g' % x[t, s].x +'\n');
            file.write('Run-time: %s' % "{:.2f}".format((time.time() - start_time)) + ' sec');
            file.write('\n\n');    
            file.write('Level 0 Capacities: ' + str(G.edges[s,1]['capacity']) +'\n\n');
            
            
            lev1 = 0;
            lev2 = 0;
            lev3 = 0;
            other_level = 0;
            file.write('\nInterdiction plan: \n');
            
            key1 = False;
            key2 = False;
            key3 = False;
                
            for i, j in G.edges: 
                if gamma[i,j].x > 0.0000001:
                    if G.edges[i,j]["level"] == 1:
                        if key1 == False:
                            file.write("Level 1:" +'\n');
                            key1 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev1 += 1;
                    elif G.edges[i,j]["level"] == 2:
                        if key2 == False:
                            file.write('Level 2:' +'\n');
                            key2 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev2 += 1;
                    elif G.edges[i,j]["level"] == 3:
                        if key3 == False:
                            file.write('Level 3:' +'\n');
                            key3 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev3 += 1;
                    else:
                        file.write('Other Levels:' +'\n');
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        other_level += 1;
                   
                        
            
            file.write('\n');
            file.write('Special arcs with flow' +'\n');    
                
            for i, j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    if x[i,j].x > 0.0000001:
                        file.write('Arc (%s,%s): ' %(i,j) + '\t Flow: %f' % x[i,j].x +'\n');
                        
                        
            file_summary.write(network[n]+'_b'+str(budget[b])+','+str(budget[b])+','+str(calA_level)+','
                               +str(t+1)+','+str(G.number_of_edges())+','+str("{:.3f}".format((OptGap)))+','+str(obj)+','
                               +str("{:.2f}".format(x[t, s].x))+','+str(lev1)+','+str(lev2)+','+str(lev3)+','
                               +str(other_level)+','+str("{:.2f}".format((time.time() - start_time)))+'sec\n');
            
            
            file.write('\n');
            file.close();
                         
file_summary.close();
            
   